In [1]:
import urllib
from collections import defaultdict
import numpy
import gzip
import string
import re
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import linear_model
import nltk
from nltk.tokenize import treebank
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestRegressor
from textblob import TextBlob

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])


In [ ]:
from math import exp 

In [2]:
posW = opinion_lexicon.positive()
negW = opinion_lexicon.negative()
score = defaultdict(int)
for w in posW:
    score[w] = 1
for w in negW:
    score[w] = -1
def demo_liu_hu_lexicon(sentence):
    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [score[word.lower()] for word in tokenizer.tokenize(sentence) ]
    #print tokenized_sent
    return sum(tokenized_sent)/(1+1.0*len(tokenized_sent))
    #print tokenized_sent
    #tokenized_sent = [word for word in tokenized_sent if word not in stopwords.words('english')]
    '''
    #print len(tokenized_sent)
    for word in tokenized_sent:
        if word in posW:
            pos_words += 1
        elif word in negW:
            neg_words += 1

    if pos_words > neg_words:
        return 1
    elif pos_words < neg_words:
        return -1
    elif pos_words == neg_words:
        return 0
    '''

In [3]:
demo_liu_hu_lexicon("Great new perspective on Ash!")

0.14285714285714285

In [4]:
print "Reading data..."
validate = list(readGz("R:/UCSD/Fall'15/CSE 255/Assignment 1/assignment1/validate_100K.json.gz"))
print "done"
validate = validate[0]


Reading data...
done


In [5]:
len(validate)

100000

In [6]:
print "Reading data..."
data = list(readGz("R:/UCSD/Fall'15/CSE 255/Assignment 1/assignment1/train_900000.json.gz"))
print "done"
print len(data)


Reading data...
done
900000


In [7]:
data_all = data[:]
data_small = data[:100000]

In [9]:
print len(data_small)

100000


In [31]:
userR = defaultdict(list)
itemR = defaultdict(list)
userReview = defaultdict(list)
itemavg = defaultdict(list)
itemT = defaultdict(list)
for d in data:
    time = d['unixReviewTime']
    itemT[d['itemID']].append(time)
    #useravg[d['reviewerID']].append(d['rating'])
    userReview[d['reviewerID']].append(d['rating'])
    itemavg[d['itemID']].append(d['rating'])
    if d['helpful']['outOf'] != 0:
        userR[d['reviewerID']].append(1.0*d['helpful']['nHelpful']/d['helpful']['outOf'])
        itemR[d['itemID']].append(1.0*d['helpful']['nHelpful']/d['helpful']['outOf'])
    else:
        userR[d['reviewerID']].append(0.0)
        itemR[d['itemID']].append(0.0)
    

In [ ]:
avgLen = 0
var = 0
for d in data:
    avgLen += len(d['reviewText'])
    var += pow(len(d['reviewText']),2)
avgLen /= len(data)
var /= len(data)
var -= pow(avgLen,2)

In [ ]:
print avgLen
print var

In [ ]:
maxO = 0
avgO = 0
count = 0
for d in data:
    if d['helpful']['outOf'] > 1000:
        count += 1
        #count += 1
        #avgO += d['helpful']['outOf']
        #maxO = max(maxO, d['helpful']['outOf'])
#print maxO
#print avgO/count
print count

In [13]:
def sent_feat(text,rating):
    blob = TextBlob(text)
    sent=0.0
    for sentence in blob.sentences:
        sent+=sentence.sentiment.polarity

    if sent>0:
        r=5-rating
    else:
        r=rating
    return abs(sent)*r

In [14]:
sent_nltk = defaultdict(float)
sent_tb = defaultdict(float)
sent_nltk_sum = defaultdict(float)
sent_tb_sum = defaultdict(float)

iter = 0
for d in data:
    iter += 1
    if iter % 10000 == 0:
        print iter
    #print iter
    sent_nltk_sum[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['summary'])
    sent_nltk[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['reviewText'])
    sent_tb[(d['reviewerID'], d['itemID'])] = sent_feat(d['reviewText'], d['rating'])
    sent_tb_sum[(d['reviewerID'], d['itemID'])] = sent_feat(d['summary'], d['rating'])

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000


In [16]:
len(sent_tb)

900000

In [17]:
iter = 0
for d in validate:
    iter += 1
    if iter % 10000 == 0:
        print iter
    sent_nltk_sum[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['summary'])
    sent_nltk[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['reviewText'])
    sent_tb[(d['reviewerID'], d['itemID'])] = sent_feat(d['reviewText'], d['rating'])
    sent_tb_sum[(d['reviewerID'], d['itemID'])] = sent_feat(d['summary'], d['rating'])

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [19]:
data_h = list(readGz("helpful.json.gz"))
print len(data_h)

50000


In [20]:
iter = 0
for d in data_h:
    iter += 1
    if iter % 10000 == 0:
        print iter
    sent_nltk_sum[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['summary'])
    sent_nltk[(d['reviewerID'], d['itemID'])] = demo_liu_hu_lexicon(d['reviewText'])
    sent_tb[(d['reviewerID'], d['itemID'])] = sent_feat(d['reviewText'], d['rating'])
    sent_tb_sum[(d['reviewerID'], d['itemID'])] = sent_feat(d['summary'], d['rating'])

10000
20000
30000
40000
50000


In [21]:
for d in validate:
    data.append(d)

In [22]:
len(data)

1000000

In [60]:
y = [1.0*d['helpful']['nHelpful']/d['helpful']['outOf'] for d in data if d['helpful']['outOf'] != 0]

In [24]:
outof = [d['helpful']['outOf'] for d in validate]
y_validate = [d['helpful']['nHelpful']  for d in validate]

In [59]:
data = data_all[:]
print len(data)

900000


In [74]:
def feature(d):
    feat = [1.0]
    
    words = re.findall(r"[\w']+",d['reviewText'])
    feat.append(len(words))
    feat.append(d['rating'])
    feat.append(d['helpful']['outOf'])
    caps = sum([1 for w in words if w.isupper()])
    feat.append(caps)
    feat.append(len(userR[d['reviewerID']]))
    feat.append(len(itemR[d['itemID']]))

    
    #feat.append(sent_tb[(d['reviewerID'], d['itemID'])])
    feat.append(sent_nltk[(d['reviewerID'], d['itemID'])])
    
    #feat.append(sum(userR[d['reviewerID']])*1.0/len(userR[d['reviewerID']]))
    feat.append(len(d['reviewText']))
    caps = sum([1 for c in d['reviewText'] if c.isupper()])
    feat.append(caps)
    
    count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
    feat.append(count(d['reviewText'], string.punctuation))
    feat.append(count(d['summary'], string.punctuation))
    
    words = re.findall(r"[\w']+",d['summary'])
    caps = sum([1 for w in words if w.isupper()])
    feat.append(caps)
    caps = sum([1 for c in d['summary'] if c.isupper()])
    feat.append(caps)
    
    if d['rating'] == 1.0:
        feat.append(1)
    else:
        feat.append(0)
    if d['rating'] == 2.0:
        feat.append(1)
    else:
        feat.append(0)
    if d['rating'] == 3.0:
        feat.append(1)
    else:
        feat.append(0)
    if d['rating'] == 4.0:
        feat.append(1)
    else:
        feat.append(0)
    if len(d['reviewText']) < 100:
        feat.append(1)
    else:
        feat.append(0)
    if len(d['reviewText']) > 1000:
        feat.append(1)
    else:
        feat.append(0)
    return feat
X = [feature(d) for d in data if d['helpful']['outOf'] != 0]
X_validate = [feature(d) for d in validate]
print "feature created"

feature created


In [53]:
X_validate = [feature(d) for d in validate]

In [78]:
#### theta,residuals,rank,s = numpy.linalg.lstsq(X, y)
regr_1 = DecisionTreeRegressor(max_depth=10)#, max_features = 7)
regr_1.fit(X, y)
#clf = linear_model.LinearRegression()
#clf.fit(X,y)
#theta = clf.coef_
#theta[0] = clf.intercept_

#estimator = RandomForestRegressor(n_estimators=100, max_depth=10)
#estimator.fit(X,y)
print "fit"

pred = regr_1.predict(X_validate)
#pred = estimator.predict(X_validate)
#print theta
#print residuals

mae = 0
#mae = sum(abs(y_validate - pred))
#'''
for i in range(len(y_validate)):
    #predict = round(inner(X_validate[i],theta)*outof[i])
    predict = round(pred[i]*outof[i])
    #predict = pred[i]*outof[i]
    mae += abs(y_validate[i] - predict)
#'''
print "mae validate: ", mae*1.0/len(y_validate)

fit
mae validate:  0.54754


In [79]:

#X_test =  [feature(d) for d in data_h]
test = {}
X_test = []
for d in data_h:
    X_test.append(feature(d))
pred_test = regr_1.predict(X_test)

predictions = open("predictions_Helpful.txt", 'w')
iter = 0
for l in open("pairs_Helpful.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    pred = int(round(outOf*pred_test[iter]))
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(pred) + '\n')
    iter += 1

predictions.close()

In [ ]:
max_h = 0
o = []
for d in data_h:
    o.append(d['helpful']['outOf'])
    max_h = max(max_h, d['helpful']['outOf'])
print max_h
print sorted(o, reverse=True)